# Подготовка данных
Необходимо создать excel или использовать готовую таблицу с некоторыми тестовыми данными

In [16]:
import pandas as pd
import math
import time

df = pd.read_excel("tests6.xlsx")
df.head()

,Id,Direction,Section,TestCaseName,Automated,Preconditions,Steps,Postconditions,ExpectedResult,TestData,Comments,Iterations,Priority,State,CreatedDate,CreatedById,Tags
0,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Highest,Ready,05/23/2024 14:48:49,Служебный пользователь для Автотестов,смоук хф
1,NaN,NaN,NaN,NaN,NaN,"Роль:\n\n\nОперационист ТП, Контролер",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,Выполнен вход в ЕФР под ролью Контролер,NaN,NaN,Открывается страница поиска клиента,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,Предварительно инициировать/найти операцию дос...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,"Кликнуть на раздел ""Задачи"" в верхней части эк...",NaN,На странице задач отображается список задач(по...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Начало обработки

Удаляем ненужные столбцы, заполняем нужные ячейки дублирующими значениями

In [17]:
start_timer = time.time()

def clean_dataset(dataset):
    output_dataset = dataset.drop(['Automated', 'Iterations', 'Priority', 'State', 'CreatedDate', 'CreatedById', 'Tags', 'Comments', 'Direction'], axis=1, inplace=False)
    # заполняем столбики значениями
    for index, row in output_dataset.iterrows():
        if math.isnan(output_dataset.loc[index, 'Id']):
            output_dataset.loc[index, 'Id'] = output_dataset.loc[index - 1, 'Id']

        if pd.isna(output_dataset.loc[index, 'Section']):
            output_dataset.loc[index, 'Section'] = output_dataset.loc[index - 1, 'Section']

        if pd.isna(output_dataset.loc[index, 'TestCaseName']):
            output_dataset.loc[index, 'TestCaseName'] = output_dataset.loc[index - 1, 'TestCaseName']

    return output_dataset

output = clean_dataset(df)
end_timer = time.time()

print(end_timer - start_timer)
output.to_excel("output.xlsx", index = False)
output.head()

0.016858577728271484


,Id,Section,TestCaseName,Preconditions,Steps,Postconditions,ExpectedResult,TestData
0,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,NaN,NaN,NaN,NaN
1,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,"Роль:\n\n\nОперационист ТП, Контролер",NaN,NaN,NaN,NaN
2,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,Выполнен вход в ЕФР под ролью Контролер,NaN,NaN,Открывается страница поиска клиента,NaN
3,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,Предварительно инициировать/найти операцию дос...,NaN,NaN,NaN,NaN
4,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Кликнуть на раздел ""Задачи"" в верхней части эк...",NaN,На странице задач отображается список задач(по...,NaN


# Разбиваем тест-кейсы на отдельные DataFrame

На выходе получим массив датафреймов(тест-кейсов)

In [18]:
start_timer = time.time()

def parse_tests_by_id(dataset):
    test_cases = []
    start_index = 0
    current_id = dataset.loc[0, 'Id']  # Текущий Id для сравнения

    for index, row in dataset.iterrows():
        if row['Id'] != current_id:
            # Добавляем срез предыдущих строк как новый DataFrame
            test_cases.append(dataset.iloc[start_index:index].copy())
            start_index = index  # Обновляем начальный индекс для нового теста
            current_id = row['Id']  # Обновляем текущий Id

    # Добавляем последний тест
    if start_index < len(dataset):
        test_cases.append(dataset.iloc[start_index:].copy())

    return test_cases

end_timer = time.time()
print(end_timer - start_timer)
output_array = parse_tests_by_id(output)
output_array[0]

0.0


,Id,Section,TestCaseName,Preconditions,Steps,Postconditions,ExpectedResult,TestData
0,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,NaN,NaN,NaN,NaN
1,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,"Роль:\n\n\nОперационист ТП, Контролер",NaN,NaN,NaN,NaN
2,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,Выполнен вход в ЕФР под ролью Контролер,NaN,NaN,Открывается страница поиска клиента,NaN
3,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,Предварительно инициировать/найти операцию дос...,NaN,NaN,NaN,NaN
4,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Кликнуть на раздел ""Задачи"" в верхней части эк...",NaN,На странице задач отображается список задач(по...,NaN
5,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Нажать ""Взять"" для задачи из пула текущего пол...",NaN,Задаче назначается исполнитель – текущий польз...,NaN
6,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Кликнуть ""Мои задачи""",NaN,"Открывается пул задач, доступных на исполнение",NaN
7,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Нажать ""Исполнить"" для задачи из пула текущего...",NaN,Открывается форма задачи в текущей вкладке,NaN


In [19]:
output_array[1]

,Id,Section,TestCaseName,Preconditions,Steps,Postconditions,ExpectedResult,TestData
8,544032.0,Пул задач пользователя,Контролер. Взять и исполнить задачу,NaN,NaN,NaN,NaN,NaN
9,544032.0,Пул задач пользователя,Контролер. Взять и исполнить задачу,Роль:\n\n\nОперационист ТП,NaN,NaN,NaN,NaN
10,544032.0,Пул задач пользователя,Контролер. Взять и исполнить задачу,Выполнен вход в ЕФР,NaN,NaN,Открывается страница поиска клиента,NaN
11,544032.0,Пул задач пользователя,Контролер. Взять и исполнить задачу,NaN,"Кликнуть ""Задачи"" в верхней части экрана",NaN,На форме отображается список задач доступных н...,NaN
12,544032.0,Пул задач пользователя,Контролер. Взять и исполнить задачу,NaN,"Нажать ""Взять и исполнить"" для задачи из пула ...",NaN,открывается форма задачи,NaN


In [20]:
output_array[2]

,Id,Section,TestCaseName,Preconditions,Steps,Postconditions,ExpectedResult,TestData
13,544033.0,Пул задач пользователя,Контролер. 2 сессии Негатив,NaN,NaN,NaN,NaN,NaN
14,544033.0,Пул задач пользователя,Контролер. 2 сессии Негатив,Роль:\n\n\nОперационист ТП\nМенеджер-операцион...,NaN,NaN,NaN,NaN
15,544033.0,Пул задач пользователя,Контролер. 2 сессии Негатив,Выполнен вход в ЕФР,NaN,NaN,Открывается страница поиска клиента,NaN
16,544033.0,Пул задач пользователя,Контролер. 2 сессии Негатив,"нажать ""Задачи""",NaN,NaN,NaN,NaN
17,544033.0,Пул задач пользователя,Контролер. 2 сессии Негатив,В другой сессии браузера авторизоваться пользо...,NaN,NaN,Открывается страница поиска клиента,NaN
18,544033.0,Пул задач пользователя,Контролер. 2 сессии Негатив,"Кликнуть ""Задачи"" в верхней части экрана",NaN,NaN,На форме отображается список задач доступных н...,NaN
19,544033.0,Пул задач пользователя,Контролер. 2 сессии Негатив,"Нажать ""Взять"" для задачи из пула текущего пол...",NaN,NaN,NaN,NaN
20,544033.0,Пул задач пользователя,Контролер. 2 сессии Негатив,NaN,Перейти на страницу входа в систему ЕФР https:...,NaN,Отображается страница входа в систему ЕФР,NaN
21,544033.0,Пул задач пользователя,Контролер. 2 сессии Негатив,NaN,Ввести логин и пароль не заблокированного поль...,NaN,Вход осуществлен успешно. Отображается страниц...,NaN
22,544033.0,Пул задач пользователя,Контролер. 2 сессии Негатив,NaN,"Кликнуть ""Задачи"" в верхней части экрана",NaN,На форме отображается список задач доступных н...,NaN


# Удаление пустых ячеек

In [24]:
start_timer = time.time()

def remove_empty_cells(array):
    # обрабатываем Preconditions (удаляем первую ячейку)
    for dframe in array:
        if pd.isna(dframe.at[dframe.index[0], 'Preconditions']):
            for i in range(len(dframe['Preconditions'])):
                if i == len(dframe.index) - 1:
                    break
                else:
                    dframe.at[dframe.index[i], 'Preconditions'] = dframe.at[dframe.index[i + 1], 'Preconditions']
                    
        if pd.isna(dframe.at[dframe.index[0], 'Steps']):
            counter = 0
            for i in range(len(dframe['Steps'])):
                if pd.isna(dframe.at[dframe.index[i], 'Steps']):
                    counter += 1
                else:
                    break
            
            for i in range(len(dframe['Steps'])):
                if i + counter == len(dframe.index) - 1:
                    break
                else:
                    dframe.at[dframe.index[i], 'Steps'] = dframe.at[dframe.index[i + counter], 'Steps']
                    
            
    return array

edited_output_array = remove_empty_cells(output_array)
end_timer = time.time()
print(end_timer - start_timer)
edited_output_array[0]

0.0


,Id,Section,TestCaseName,Preconditions,Steps,Postconditions,ExpectedResult,TestData
0,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,"Роль:\n\n\nОперационист ТП, Контролер","Кликнуть на раздел ""Задачи"" в верхней части эк...",NaN,NaN,NaN
1,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,Выполнен вход в ЕФР под ролью Контролер,"Нажать ""Взять"" для задачи из пула текущего пол...",NaN,NaN,NaN
2,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,Предварительно инициировать/найти операцию дос...,"Кликнуть ""Мои задачи""",NaN,Открывается страница поиска клиента,NaN
3,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,NaN,NaN,NaN,NaN
4,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Кликнуть на раздел ""Задачи"" в верхней части эк...",NaN,На странице задач отображается список задач(по...,NaN
5,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Нажать ""Взять"" для задачи из пула текущего пол...",NaN,Задаче назначается исполнитель – текущий польз...,NaN
6,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Кликнуть ""Мои задачи""",NaN,"Открывается пул задач, доступных на исполнение",NaN
7,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Нажать ""Исполнить"" для задачи из пула текущего...",NaN,Открывается форма задачи в текущей вкладке,NaN


# spaCy

```conda install spacy```

Тест маленькой модели

In [22]:
import spacy

# запуск таймера
start_timer = time.time()
# подгрузка языковой модели
nlp = spacy.load("ru_core_news_lg")

# функция векторизации
def vectorize_text(text, model):
    doc = model(text)
    return doc.vector

print(str(df))

vector_table = vectorize_text(str(df), nlp)

print(vector_table)
# vector_id = vectorize_text(str(df['Id']), nlp)
# vector_direction = vectorize_text(str(df['Direction']), nlp)
# vector_steps = vectorize_text(str(df['Steps']), nlp)
# 
# print(str(df))
# 
# print(str(df['Id']), "\n")
# print(str(df['Direction']), "\n")
# print(str(df['Steps']), "\n")
# print(type(vector_id))
# 
# print(vector_id, "\n")
# print(vector_direction, "\n")
# print(vector_steps, "\n")

end_timer = time.time()
print(end_timer - start_timer)

          Id   Direction                 Section  \
0   544031.0  AI_Отладка  Пул задач пользователя   
1        NaN         NaN                     NaN   
2        NaN         NaN                     NaN   
3        NaN         NaN                     NaN   
4        NaN         NaN                     NaN   
..       ...         ...                     ...   
59       NaN         NaN                     NaN   
60       NaN         NaN                     NaN   
61       NaN         NaN                     NaN   
62       NaN         NaN                     NaN   
63       NaN         NaN                     NaN   

                    TestCaseName Automated  \
0   Контролер. Исполнить задачу      False   
1                            NaN       NaN   
2                            NaN       NaN   
3                            NaN       NaN   
4                            NaN       NaN   
..                           ...       ...   
59                           NaN       NaN   
60     

In [23]:
#print(len(vector_id), len(vector_direction), len(vector_steps))
print(len(vector_table))

300
